## Imports

In [ ]:
import pandas as pd
import numpy as np

In [36]:
df = pd.read_csv(
    'H1b data/h1b_data2024-25.csv',
    engine='python',     # more tolerant than the C engine
    sep=None,            # sniff the delimiter (comma/semicolon/tab)
    encoding='utf-16',  # file is UTF-16 encoded
    on_bad_lines='warn', # show which lines are malformed (pandas >=1.3)
)


## Intro data

In [37]:
df.head(20)

,Line by line,Fiscal Year,Employer (Petitioner) Name,Tax ID,Industry (NAICS) Code,Petitioner City,Petitioner State,Petitioner Zip Code,New Employment Approval,New Employment Denial,Continuation Approval,Continuation Denial,Change with Same Employer Approval,Change with Same Employer Denial,New Concurrent Approval,New Concurrent Denial,Change of Employer Approval,Change of Employer Denial,Amended Approval,Amended Denial
0,1,2025,NaN,NaN,"54 - Professional, Scientific, and Technical S...",FRISCO,TX,75034.0,0,0,0,0,0,0,0,0,1,0,0,0
1,2,2025,NaN,274.0,"54 - Professional, Scientific, and Technical S...",GUILFORD,CT,6437.0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,2025,NaN,305.0,"54 - Professional, Scientific, and Technical S...",WASHINGTON,DC,20001.0,0,0,1,0,0,0,0,0,0,0,0,0
3,4,2025,NaN,3188.0,"54 - Professional, Scientific, and Technical S...",ARLINGTON,TX,76011.0,0,0,0,0,1,0,0,0,0,0,0,0
4,5,2025,NaN,4668.0,"54 - Professional, Scientific, and Technical S...",CHICAGO,IL,60625.0,2,0,0,0,0,0,0,0,0,0,0,0
5,6,2025,NaN,5680.0,"54 - Professional, Scientific, and Technical S...",BOSTON,MA,2110.0,1,0,0,0,0,0,0,0,0,0,0,0
6,7,2025,NaN,6264.0,"54 - Professional, Scientific, and Technical S...",HOUSTON,TX,77027.0,1,0,0,0,0,0,0,0,0,0,0,0
7,8,2025,NaN,6473.0,72 - Accommodation and Food Services,TAOS,NM,87571.0,1,0,0,0,0,0,0,0,0,0,0,0
8,9,2025,NaN,7191.0,"54 - Professional, Scientific, and Technical S...",HENDERSON,NV,89012.0,1,0,0,0,0,0,0,0,0,0,0,0
9,10,2025,NaN,7351.0,"54 - Professional, Scientific, and Technical S...",NEW YORK,NY,10006.0,0,0,0,0,1,0,0,0,0,0,0,0


Employer name listed in alphabetical order. 

**Initial thoughts to clean:**
- remove null /blank Employer Name
- Find definitions of the types of approval/denial
- look at which are niche categories that can be effectively ignored for the initial data question (small employers most likely)



In [38]:
df.shape

(115677, 20)

### Definitions of Columns (drop down):

- **Fiscal Year:** The fiscal year in which USCIS first recorded an approval or denial in the electronic systems. USCIS follows the U.S. Federal Government fiscal year (FY) calendar, so data sets presented by fiscal year cover Oct. 1 of one year to Sept. 30 of the next year. Applications and petitions USCIS receives on a given date are generally adjudicated on a later date. Therefore, data in the H-1B Employer Data Hub reflect the date we adjudicated the application or petition rather than the date received.
- **Employer Name:** Petitioner’s firm/employer name from I-129, Page 1, Part 1, Question 2 of the current form.
- **NAICS Code:** A character string that stands for an industry classification within the North American Industry Classification System from the Form I-129, Page 19, Section 1, Question 6. Data are presented at the two-digit level. For more information on the NAICS, visit the U.S. Census Bureau’s North American Industry Classification Code webpage.  NAICS code 99 means the industry is unknown. Any petition that had a blank code was assigned as 99 as well.
- **Tax ID:** The last four digits of the Petitioner’s Tax ID Number from the Form I-129, Page 1, Part 1, Question 5.
- **State:** Petitioner’s state from the Form I-129, Page 1, Part 1, Question 3. This is the State indicated in the mailing address of the employer and is not necessarily the beneficiary(ies) work location
- **City:**	Petitioner’s city from the Form I-129, Page 1, Part 1, Question 3. This is the City indicated in the mailing address of the employer and is not necessarily the beneficiary(ies) work location.
- **ZIP:** 	Petitioner’s five-digit ZIP code from the Form I-129, Petition for a Nonimmigrant Worker, Page 1, Part 1, Question 3. This is the ZIP code indicated in the mailing address of the employer and is not necessarily the beneficiary(ies) work location.
- **New Employment Approval:** Initially approved H-1B petitions with “New employment” selected on Part 2, Question 2 of the Form I-129. Number represents counts of workers approved. The beneficiary:
    1. Is outside the United States and holds no classification;
    2. Will begin employment in the United States for a new employer in a different nonimmigrant classification than the beneficiary currently holds; or
    3. Will work for the same employer but in a different nonimmigrant classification.
    
- **New Employment Denial**: Initially denied H-1B petitions with “New employment” selected on Part 2, Question 2 of the Form I-129. Number represents counts of workers denied.
- **Continuation Approval**: Approved H-1B petitions with “Continuation of previously approved employment without change with the same employer” selected on Part 2, Question 2 of the Form I-129. The beneficiary will continue to work in the same nonimmigrant classification currently held with no change to the employment. Number represents counts of workers approved.
- **Continuation Denial**: Denied H-1B petitions with “Continuation of previously approved employment without change with the same employer” selected on Part 2, Question 2 of the Form I-129.The beneficiary will be denied continuing employment in the same nonimmigrant classification currently held with no change to the employment. Number represents counts of workers denied.
- **Change with Same Employer Approval**: Approved H-1B petitions with “Change in previously approved employment” selected on Part 2, Question 2 of the Form I-129. This selection is used to notify USCIS of a non-material change to the previously approved employment, such as a change in job title without a material change in job duties. Number represents counts of workers approved.
- **Change with Same Employer Denial**:	Denied H-1B petitions with “Change in previously approved employment” selected on Part 2, Question 2 of the Form I-129 are those who applied and were denied a non-material change to the previously approved employment, such as a change in job title without a material change in job duties. Number represents counts of workers denied.
- **New Concurrent Approval** Approved H-1B petitions with “New concurrent employment” selected on Part 2, Question 2 of the Form I-129. The beneficiary will begin new employment with an additional employer in the same nonimmigrant classification the beneficiary currently holds while continuing to work for his or her current employer in the same classification. Number represents counts of workers approved.
- **New Concurrent Denial**: Denied H-1B petitions with “New concurrent employment” selected on Part 2, Question 2 of the Form I-129 are those denied a petition to begin new employment with an additional employer in the same nonimmigrant classification the beneficiary currently holds. The beneficiary would have continued working for his or her current employer in the same classification. Number represents counts of workers denied.
- **Change of Employer Approval:** Approved H -1B petitions with “Change of employer” selected on Part 2, Question 2 of the Form I-129. The beneficiary will begin employment working for a new employer in the same nonimmigrant classification that the beneficiary currently holds. Number represents counts of workers approved.
- **Change of Employer Denial:** Denied H-1B petitions with “Change of employer” selected on Part 2, Question 2 of the Form I-129 are those who applied and were denied beginning employment working for a new employer in the same nonimmigrant classification that the beneficiary currently holds. Number represents counts of workers denied.
- **Amended Approval** Approved H-1B petitions with “Amended petition” selected on Part 2, Question 2 of the Form I-129.  This selection is used to notify USCIS of a material change in the terms or conditions of employment or the beneficiary's eligibility as specified in the original approved petition, or to request substitutions. Number represents counts of workers approved.
- **Amended Denial** Denied H -1B petitions with “Amended petition” are those applied for and were denied selected on Part 2, Question 2 of the Form I-129 a material change in the terms or conditions of employment, or the beneficiary's eligibility as specified in the original approved petition. Number represents counts of workers denied.


### Initial EDA and goals

1. Identify NULL in Employer Name and remove if small
2. Summary data for the types of petition decisions
3. Identify largest 'insourcers' since 2023, any trends

We'll need to do summary statistics for the nnumber of workers approved/denied etc in each record, first checking dtypes potentially converting to numbers

In [39]:
# check data types in the dataframe
df.dtypes

Line by line                           object
Fiscal Year                             int64
Employer (Petitioner) Name             object
Tax ID                                float64
Industry (NAICS) Code                  object
Petitioner City                        object
Petitioner State                       object
Petitioner Zip Code                   float64
New Employment Approval                object
New Employment Denial                   int64
Continuation Approval                  object
Continuation Denial                     int64
Change with Same Employer Approval     object
Change with Same Employer Denial        int64
New Concurrent Approval                 int64
New Concurrent Denial                   int64
Change of Employer Approval            object
Change of Employer Denial               int64
Amended Approval                       object
Amended Denial                          int64
dtype: object

### Clean up Columns dtype & format

In [40]:
df.columns

Index(['Line by line', 'Fiscal Year   ', 'Employer (Petitioner) Name',
       'Tax ID', 'Industry (NAICS) Code', 'Petitioner City',
       'Petitioner State', 'Petitioner Zip Code', 'New Employment Approval',
       'New Employment Denial', 'Continuation Approval', 'Continuation Denial',
       'Change with Same Employer Approval',
       'Change with Same Employer Denial', 'New Concurrent Approval',
       'New Concurrent Denial', 'Change of Employer Approval',
       'Change of Employer Denial', 'Amended Approval', 'Amended Denial'],
      dtype='object')

In [41]:
print(df.columns.tolist())

['Line by line', 'Fiscal Year   ', 'Employer (Petitioner) Name', 'Tax ID', 'Industry (NAICS) Code', 'Petitioner City', 'Petitioner State', 'Petitioner Zip Code', 'New Employment Approval', 'New Employment Denial', 'Continuation Approval', 'Continuation Denial', 'Change with Same Employer Approval', 'Change with Same Employer Denial', 'New Concurrent Approval', 'New Concurrent Denial', 'Change of Employer Approval', 'Change of Employer Denial', 'Amended Approval', 'Amended Denial']


Strip whitespace

In [42]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace('\xa0', ' ', regex=True)  # replace non-breaking spaces


In [46]:
df.columns.tolist()

['Line by line',
 'Fiscal Year',
 'Employer (Petitioner) Name',
 'Tax ID',
 'Industry (NAICS) Code',
 'Petitioner City',
 'Petitioner State',
 'Petitioner Zip Code',
 'New Employment Approval',
 'New Employment Denial',
 'Continuation Approval',
 'Continuation Denial',
 'Change with Same Employer Approval',
 'Change with Same Employer Denial',
 'New Concurrent Approval',
 'New Concurrent Denial',
 'Change of Employer Approval',
 'Change of Employer Denial',
 'Amended Approval',
 'Amended Denial']

In [47]:
# change dtype to integer for relevant columns
df['New Employment Approval'] = pd.to_numeric(df['New Employment Approval'], errors='coerce').astype('Int64')
df['New Employment Denial'] = pd.to_numeric(df['New Employment Denial'], errors='coerce').astype('Int64')
df['Continuation Approval'] = pd.to_numeric(df['Continuation Approval'], errors='coerce').astype('Int64')
df['Continuation Denial'] = pd.to_numeric(df['Continuation Denial'], errors='coerce').astype('Int64')
df['Change with Same Employer Approval'] = pd.to_numeric(df['Change with Same Employer Approval'], errors='coerce').astype('Int64')
df['Change with Same Employer Denial'] = pd.to_numeric(df['Change with Same Employer Denial'], errors='coerce').astype('Int64')
df['New Concurrent Approval'] = pd.to_numeric(df['New Concurrent Approval'], errors='coerce').astype('Int64')
df['New Concurrent Denial'] = pd.to_numeric(df['New Concurrent Denial'], errors='coerce').astype('Int64')
df['Change of Employer Approval'] = pd.to_numeric(df['Change of Employer Approval'], errors='coerce').astype('Int64')
df['Change of Employer Denial'] = pd.to_numeric(df['Change of Employer Denial'], errors='coerce').astype('Int64')
df['Amended Approval'] = pd.to_numeric(df['Amended Approval'], errors='coerce').astype('Int64')
df['Amended Denial'] = pd.to_numeric(df['Amended Denial'], errors='coerce').astype('Int64')

In [48]:
df.dtypes

Line by line                           object
Fiscal Year                             int64
Employer (Petitioner) Name             object
Tax ID                                float64
Industry (NAICS) Code                  object
Petitioner City                        object
Petitioner State                       object
Petitioner Zip Code                   float64
New Employment Approval                 Int64
New Employment Denial                   Int64
Continuation Approval                   Int64
Continuation Denial                     Int64
Change with Same Employer Approval      Int64
Change with Same Employer Denial        Int64
New Concurrent Approval                 Int64
New Concurrent Denial                   Int64
Change of Employer Approval             Int64
Change of Employer Denial               Int64
Amended Approval                        Int64
Amended Denial                          Int64
dtype: object

### Remove null employer names

In [53]:
df['Employer (Petitioner) Name'].isnull().sum()

12

only 12 Nulls in whole dataframe

In [54]:
df[df['Employer (Petitioner) Name'].isnull()]

,Line by line,Fiscal Year,Employer (Petitioner) Name,Tax ID,Industry (NAICS) Code,Petitioner City,Petitioner State,Petitioner Zip Code,New Employment Approval,New Employment Denial,Continuation Approval,Continuation Denial,Change with Same Employer Approval,Change with Same Employer Denial,New Concurrent Approval,New Concurrent Denial,Change of Employer Approval,Change of Employer Denial,Amended Approval,Amended Denial
0,1,2025,NaN,NaN,"54 - Professional, Scientific, and Technical S...",FRISCO,TX,75034.0,0,0,0,0,0,0,0,0,1,0,0,0
1,2,2025,NaN,274.0,"54 - Professional, Scientific, and Technical S...",GUILFORD,CT,6437.0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,2025,NaN,305.0,"54 - Professional, Scientific, and Technical S...",WASHINGTON,DC,20001.0,0,0,1,0,0,0,0,0,0,0,0,0
3,4,2025,NaN,3188.0,"54 - Professional, Scientific, and Technical S...",ARLINGTON,TX,76011.0,0,0,0,0,1,0,0,0,0,0,0,0
4,5,2025,NaN,4668.0,"54 - Professional, Scientific, and Technical S...",CHICAGO,IL,60625.0,2,0,0,0,0,0,0,0,0,0,0,0
5,6,2025,NaN,5680.0,"54 - Professional, Scientific, and Technical S...",BOSTON,MA,2110.0,1,0,0,0,0,0,0,0,0,0,0,0
6,7,2025,NaN,6264.0,"54 - Professional, Scientific, and Technical S...",HOUSTON,TX,77027.0,1,0,0,0,0,0,0,0,0,0,0,0
7,8,2025,NaN,6473.0,72 - Accommodation and Food Services,TAOS,NM,87571.0,1,0,0,0,0,0,0,0,0,0,0,0
8,9,2025,NaN,7191.0,"54 - Professional, Scientific, and Technical S...",HENDERSON,NV,89012.0,1,0,0,0,0,0,0,0,0,0,0,0
9,10,2025,NaN,7351.0,"54 - Professional, Scientific, and Technical S...",NEW YORK,NY,10006.0,0,0,0,0,1,0,0,0,0,0,0,0


Barely 15 workers between the 12 here in one year. Drop